# №8. Сохранение и загрузка модели

In [ ]:
import torch
import json
from pathlib import Path

## 1. Сохранение

**Примечания**:
- Не стоит сохранять модель в "лоб" `torch.save(my_model, path)`, так как при смене версии Python\ОС могут возникнуть сложности и ошибки при загрузке.
- Лучше всего сохранять ***состояние модели***, т.е. `torch.save(my_model.state_dict(), path)`.
- Можно сохранять любую необходимую информацию, например:
    * Текущие состояния `optim`, `lr_scheduler`, `loss_model`;
    * Изменения `lr`, `loss`, `acc` и прочих параметров и метрик;
    * Информация о эпохе (всего запланировано, сколько пройдено, когда сохранялись модели)
    * Код создания модели;
    * *И много другой информации*.

Пример словаря-чекпоинта для сохранения:
```python
checkpoint = {
    'info': str_info,
    'state_model': model.state_dict(),
    'state_opt': opt.state_dict(),
    'state_lr_scheduler': lr_scheduler.state_dict(),
    'loss': {
        'train_loss': train_loss,
        'val_loss': val_loss,
        'best_loss': best_loss
    },
    'metric': {
        'train_acc': train_acc,
        'val_acc': val_acc,
    },
    'lr': lr_list,
    'epoch': {
        'EPOCHS': EPOCHS,
        'save_epoch': save_epoch
    }
}
torch.save(checkpoint, 'model_state_dict_01_01_24.pt')
```

In [ ]:
# Класс для управления обучением
class TrainingTracker:
    def __init__(self, experiment_name, save_dir='./checkpoints'):
        self.experiment_name = experiment_name
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)
        
        self.history = {
            'train_loss': [], 'val_loss': [],
            'train_acc': [], 'val_acc': [],
            'learning_rates': [],
            'epoch_times': [],
            'grad_norms': []
        }
        
        self.best_metrics = {
            'best_loss': float('inf'),
            'best_accuracy': 0.0,
            'best_epoch': 0
        }
    
    def update_epoch(self, epoch, train_loss, val_loss, train_acc, val_acc, lr, epoch_time, grad_norm=None):
        self.history['train_loss'].append(train_loss)
        self.history['val_loss'].append(val_loss)
        self.history['train_acc'].append(train_acc)
        self.history['val_acc'].append(val_acc)
        self.history['learning_rates'].append(lr)
        self.history['epoch_times'].append(epoch_time)
        
        if grad_norm:
            self.history['grad_norms'].append(grad_norm)
        
        # Обновляем лучшие метрики
        if val_acc > self.best_metrics['best_accuracy']:
            self.best_metrics.update({
                'best_accuracy': val_acc,
                'best_epoch': epoch,
                'best_loss': val_loss
            })
    
    def save_checkpoint(self, model, optimizer, scheduler, epoch, config):
        checkpoint = {
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
            'scheduler_state': scheduler.state_dict() if scheduler else None,
            'tracker_history': self.history,
            'best_metrics': self.best_metrics,
            'config': config,
            'timestamp': datetime.datetime.now().isoformat()
        }
        
        filename = self.save_dir / f"{self.experiment_name}_epoch_{epoch}.pt"
        torch.save(checkpoint, filename)
        
        # Сохраняем также как best если это лучшая модель
        if epoch == self.best_metrics['best_epoch']:
            best_filename = self.save_dir / f"{self.experiment_name}_best.pt"
            torch.save(checkpoint, best_filename)
        
        # Сохраняем историю в JSON для анализа
        self.save_history_json()
    
    def save_history_json(self):
        history_file = self.save_dir / f"{self.experiment_name}_history.json"
        with open(history_file, 'w') as f:
            json.dump(self.history, f, indent=2)

In [ ]:
# Использование в тренировочном цикле
def train_model(model, train_loader, val_loader, config):
    tracker = TrainingTracker(config['experiment_name'])
    
    for epoch in range(config['epochs']):
        start_time = time.time()
        
        # Тренировка
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler)
        
        # Валидация
        val_loss, val_acc = validate_epoch(model, val_loader)
        
        # Расчет времени и градиентов
        epoch_time = time.time() - start_time
        current_lr = get_current_lr(optimizer)
        
        # Обновление трекера
        tracker.update_epoch(
            epoch, train_loss, val_loss, train_acc, val_acc, 
            current_lr, epoch_time
        )
        
        # Сохранение чекпоинта
        if epoch % config['save_every'] == 0:
            tracker.save_checkpoint(model, optimizer, scheduler, epoch, config)
        
        # Логирование
        print(f"Epoch {epoch}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    return tracker

## 2. Загрузка

**Примечание**:

- Обратить внимание на то, где хранилась модель и прочие данные во время сохранения: если модель хранилась на `cuda`, то при ее сохранении и последущей загрузке она сразу же будет автоматически пытаться загрузиться на `cuda`. Если `cuda` будет недоступна, то получим ошибку:
```
RuntimeError: Attempting to deserialize object on a CUDA devivce but torch.cuda.is_available() is False. ...
```

- Для избежания ошибки следует загружать данные с указанием параметра `map_location`:
```python
device = 'cuda' if torch.cuda.is_available() else 'cpu'
param_model = torch.load('path/to/model', map_location=device)
```

In [ ]:
def load_checkpoint(filepath, model, optimizer=None, scheduler=None, device='cuda'):
    """
    Загрузка checkpoint и восстановление состояния
    """
    checkpoint = torch.load(filepath, map_location=device)
    
    # Загрузка модели
    if 'state_model' in checkpoint:
        model.load_state_dict(checkpoint['state_model'])
    elif 'model_state' in checkpoint:
        model.load_state_dict(checkpoint['model_state'])
    elif 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'])
    
    # Загрузка оптимизатора
    if optimizer and 'state_opt' in checkpoint:
        optimizer.load_state_dict(checkpoint['state_opt'])
    elif optimizer and 'optimizer_state' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state'])
    
    # Загрузка scheduler
    if scheduler and 'state_lr_scheduler' in checkpoint:
        scheduler.load_state_dict(checkpoint['state_lr_scheduler'])
    elif scheduler and 'scheduler_state' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state'])
    
    print(f"Checkpoint loaded from {filepath}")
    print(f"Epoch: {checkpoint.get('epoch', 'N/A')}")
    
    return checkpoint

# Использование
model = YourModel()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)

checkpoint = load_checkpoint(
    'model_state_dict_01_01_24.pt', 
    model, optimizer, scheduler
)